In [0]:
import numpy as np, sys
np.random.seed(1)

from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

images, labels = (x_train[0:1000].reshape(1000,28*28) / 255,
                  y_train[0:1000])


one_hot_labels = np.zeros((len(labels),10))
for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels

test_images = x_test.reshape(len(x_test),28*28) / 255
test_labels = np.zeros((len(y_test),10))
for i,l in enumerate(y_test):
    test_labels[i][l] = 1

def tanh(x):
    return np.tanh(x)

def tanh2deriv(output):
    return 1 - (output ** 2)

def softmax(x):
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims=True)

alpha, iterations = (2, 300)
pixels_per_image, num_labels = (784, 10)
batch_size = 128

input_rows = 28
input_cols = 28

kernel_rows = 3
kernel_cols = 3
num_kernels = 16

hidden_size = ((input_rows - kernel_rows) *               #hidden_size: 10000
               (input_cols - kernel_cols)) * num_kernels

In [0]:
kernels = 0.02*np.random.random((kernel_rows*kernel_cols, #kernels.shape:(9, 16)
                                 num_kernels))-0.01

weights_1_2 = 0.2*np.random.random((hidden_size,          #weights_1_2.shape: (10000, 10)
                                    num_labels)) - 0.1

def get_image_section(layer,row_from, row_to, col_from, col_to):
    section = layer[:,row_from:row_to,col_from:col_to]
    return section.reshape(-1,1,row_to-row_from, col_to-col_from)

for j in range(iterations):
    correct_cnt = 0
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end=((i * batch_size),((i+1)*batch_size))
        layer_0 = images[batch_start:batch_end]           #layer_0.shape: (128, 784)
        
        layer_0 = layer_0.reshape(layer_0.shape[0],28,28)
        layer_0.shape                                     #layer_0.shape: (128, 28, 28)
        
        sects = list()
        for row_start in range(layer_0.shape[1]-kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0,         #sect.shape -> (128, 1, 3, 3)
                                         row_start,
                                         row_start+kernel_rows,
                                         col_start,
                                         col_start+kernel_cols)
                
               # sect sample:
               #  [[[0.95294118 0.98823529 0.88627451]
               #    [0.98823529 0.98823529 0.99215686]
               #    [0.98823529 0.98823529 0.99215686]]]
                
                
                sects.append(sect)                       #max sect len = 652 (25x25)    


        expanded_input = np.concatenate(sects,axis=1)    #max expanded_input length = 128                                                
        es = expanded_input.shape                        #expanded_input.shape (128, 625, 3, 3)
        flattened_input = expanded_input.reshape(es[0]*es[1],-1) #flattened_input.shape: (80000, 9)

        kernel_output = flattened_input.dot(kernels)     #kernel_output.shape: (80000, 16)

        layer_1 = tanh(kernel_output.reshape(es[0],-1))  #layer_1.shape: (128, 10000)
        dropout_mask = np.random.randint(2,size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = softmax(np.dot(layer_1,weights_1_2))   #layer_2.shape: (128, 10)

        #layer_2[0] sample:
        #  [0.0975271  0.1175847  0.09068662 0.09699426 0.09819915 0.09174087
        #   0.1027674  0.09465814 0.10095851 0.10888325]
        

        for k in range(batch_size):
            labelset = labels[batch_start+k:batch_start+k+1]
            _inc = int(np.argmax(layer_2[k:k+1]) == 
                               np.argmax(labelset))
            correct_cnt += _inc

        layer_2_delta = (labels[batch_start:batch_end]-layer_2)\
                        / (batch_size * layer_2.shape[0])
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * \
                        tanh2deriv(layer_1)
        layer_1_delta *= dropout_mask
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        l1d_reshape = layer_1_delta.reshape(kernel_output.shape)
        k_update = flattened_input.T.dot(l1d_reshape)
        kernels -= alpha * k_update
    
    test_correct_cnt = 0

    for i in range(len(test_images)):

        layer_0 = test_images[i:i+1]                    #layer_0.shape: (1, 784)
        layer_0 = layer_0.reshape(layer_0.shape[0],28,28)
        layer_0.shape                                   #layer_0.shape: (1, 28, 28)

        sects = list()
        for row_start in range(layer_0.shape[1]-kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0,       #sect.shape: (1, 1, 3, 3)
                                         row_start,
                                         row_start+kernel_rows,
                                         col_start,
                                         col_start+kernel_cols)
                
                # sect sample:
                #[[[[0.81960784 0.99215686 0.99215686]
                #   [0.96862745 0.99215686 0.77647059]
                #   [0.99215686 0.90588235 0.24705882]]]]
                
                
                sects.append(sect)                      #max sects len = 625 (25x25)

        expanded_input = np.concatenate(sects,axis=1)   #max expanded_input length = 1   
        es = expanded_input.shape                       #expanded_input.shape: (1, 625, 3, 3)
        flattened_input = expanded_input.reshape(es[0]*es[1],-1) #flattened_input.shape: (625, 9)
        kernel_output = flattened_input.dot(kernels)    #kernel_output.shape: (625, 16)
        layer_1 = tanh(kernel_output.reshape(es[0],-1)) #layer_1.shape: (1, 10000)
        layer_2 = np.dot(layer_1,weights_1_2)           #layer_2.shape: (1, 10)

        #layer_2 sample:
        # [[-0.04655149 -0.04742587 -0.09474036 -0.1231191   0.06017885 -0.03745107
        #   -0.05922888 -0.0845719  -0.10268217  0.04391817]]

        test_correct_cnt += int(np.argmax(layer_2) == 
                                np.argmax(test_labels[i:i+1]))
    if(j % 1 == 0):
        sys.stdout.write("\n"+ \
         "I:" + str(j) + \
         " Test-Acc:"+str(test_correct_cnt/float(len(test_images)))+\
         " Train-Acc:" + str(correct_cnt/float(len(images))))


I:0 Test-Acc:0.0162 Train-Acc:0.065
I:1 Test-Acc:0.0265 Train-Acc:0.052